In [ ]:
'''
La derniere étape de notre projet est d'utiliser les modèles de machine learning choisi pour essayer de prédire 
les données de 2022 puis de comparer les écarts avec le reel.
'''

In [127]:
# Étape 1 : Preprocessing et feature engineering de la base de donnée 2022 pour la rendre exploitable.

# Import des modules et fichiers.
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

df_2022_full=pd.read_csv('../data/world-hapiness-report-2022.csv',sep=";")

# Suppression de toutes les lignes non relatives à 2022.
df_2022=df_2022_full[(df_2022_full['year']==2022)]

# Ajout des continents.
df_continents=pd.read_csv('../data/pays&continents.csv',sep=';')
df_2022=df_2022.merge(right=df_continents,on='Country name',how='outer')
df_2022.insert(1,'Regional indicator',df_2022.pop('Regional indicator'))

# Suppression des valeurs manquantes.
df_2022=df_2022.dropna()

# Suppression de la colonne des années.
df_2022=df_2022.drop(['year'],axis=1)

# Copie du jeux de données.
df_2022_continents = df_2022.copy()

# Séparation des variables explicatives et de la variable nom de pays de la variable cible.
X_test2022_continents=df_2022_continents.drop(['Life Ladder', 'Country name'],axis=1)
y_test2022=df_2022_continents['Life Ladder']

#On encode les continents de X_2022
X_test2022_continents=pd.get_dummies(X_test2022_continents,columns=['Regional indicator'])

# Normalisation des données.
scaler=MinMaxScaler()
cols=['Log GDP per capita','Social support','Healthy life expectancy at birth','Freedom to make life choices',
      'Generosity','Perceptions of corruption','Positive affect','Negative affect']
X_test2022_continents[cols]=scaler.fit_transform(X_test2022_continents[cols])

# Suppression de la colonne "Regional indicator_Central and Eastern Europe" pour correspondre à l'encodage des données
# fait par le OneHotEncoding.
X_test2022_continents=X_test2022_continents.drop(['Regional indicator_Central and Eastern Europe'],axis=1)

# X_test2022_continents et y_test2022 sont alors respectivements les jeux d'entraînement et de test.

In [128]:
# Étape 2 : Récupération des jeux d'entraînement et de test générés dans les notebooks précédents (données de 2006 
# à 2021).

# Import des données.
X_train=pd.read_csv('../data/X_train.csv')
X_test=pd.read_csv('../data/X_test.csv')
y_train=pd.read_csv('../data/y_train.csv')
y_test=pd.read_csv('../data/y_test.csv')

# Concaténation.
X_train2021=pd.concat([X_train,X_test],axis=0)
y_train2021=pd.concat([y_train,y_test],axis=0)

# Transofrmation du jeu y en tableau python.
y_train2021=y_train2021['Life Ladder'].values

# Normalisation des données.
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
cols=['Log GDP per capita','Social support','Healthy life expectancy at birth','Freedom to make life choices',
      'Generosity', 'Perceptions of corruption','Positive affect','Negative affect']
X_train2021[cols]=scaler.fit_transform(X_train2021[cols])


In [129]:
# Étape 3: Faire tourner le modele de Random forest pour la regression et le tester.

from sklearn.ensemble import RandomForestRegressor
rf_model = RandomForestRegressor(max_depth=20,max_features='sqrt',min_samples_leaf=1,min_samples_split=2,n_estimators=300)

rf_model.fit(X_train2021,y_train2021)

print("Score sur jeu d'entrainement 2021 :",rf_model.score(X_train2021,y_train2021))
print("Score sur jeu de test 2022 :",rf_model.score(X_test2022_continents,y_test2022))

Score sur jeu d'entrainement 2021 : 0.9855614987002708
Score sur jeu de test 2022 : 0.7988966219083594


In [130]:
# Réalisation d'une prédiction du Life Ladder de 2022 par pays grâce au modèle Random forest.
y_predic2022_continents = rf_model.predict(X_test2022_continents)

# Arrondissement de la prédictions à 2 décimales.
import numpy as np
y_predic2022_continents=np.round(y_predic2022_continents,2)

# Ajout d'une colonne avec les prédictions au jeu de données df_2022.
df_2022_continents['Predictions']=y_predic2022_continents
df_2022_continents=df_2022_continents.drop(cols,axis=1)

# Ajout d'une colonne pour représenter les variations en pourcentages.
df_2022_continents['Ecart en %']=np.round((df_2022_continents['Predictions']-df_2022_continents['Life Ladder'])/df_2022_continents['Life Ladder']*100,1)

In [131]:
df_2022_continents.head(10)

,Country name,Regional indicator,Life Ladder,Predictions,Ecart en %
1,Albania,Central and Eastern Europe,5.21,4.94,-5.2
2,Argentina,Latin America and Caribbean,6.26,6.11,-2.4
3,Armenia,Commonwealth of Independent States,5.38,4.98,-7.4
4,Australia,North America and ANZ,7.04,7.03,-0.1
5,Austria,Western Europe,7.00,6.75,-3.6
6,Bangladesh,South Asia,3.41,4.08,19.6
7,Belgium,Western Europe,6.86,7.01,2.2
8,Benin,Sub-Saharan Africa,4.22,3.82,-9.5
9,Bolivia,Latin America and Caribbean,5.93,5.34,-9.9
10,Botswana,Sub-Saharan Africa,3.44,4.62,34.3


In [132]:
df_2022_continents.drop('Country name',axis=1).groupby('Regional indicator').agg('mean').sort_values('Ecart en %',ascending=True)

# On constate que le modèle prédit plutot bien avec toutefois des écarts plus importants pour les valeurs extremes.

,Life Ladder,Predictions,Ecart en %
Regional indicator,,,
Commonwealth of Independent States,5.592500,5.190000,-6.925000
East Asia,5.973333,5.570000,-6.866667
Southeast Asia,5.530000,5.115000,-6.775000
Latin America and Caribbean,6.211111,5.797222,-6.511111
Central and Eastern Europe,6.044545,5.752727,-4.500000
North America and ANZ,6.907500,6.767500,-2.100000
Western Europe,6.893333,6.808333,-1.277778
South Asia,4.270000,4.156667,0.900000
Sub-Saharan Africa,4.289643,4.256786,1.739286


In [133]:
display(df_2022_continents.sort_values('Life Ladder',ascending=True).head(10))
display(df_2022_continents.sort_values('Life Ladder',ascending=True).tail(10))

,Country name,Regional indicator,Life Ladder,Predictions,Ecart en %
61,Lebanon,Middle East and North Africa,2.35,3.89,65.5
93,Sierra Leone,Sub-Saharan Africa,2.56,3.55,38.7
21,Congo (Kinshasa),Sub-Saharan Africa,3.21,4.15,29.3
113,Zimbabwe,Sub-Saharan Africa,3.30,4.10,24.2
66,Malawi,Sub-Saharan Africa,3.36,3.71,10.4
6,Bangladesh,South Asia,3.41,4.08,19.6
10,Botswana,Sub-Saharan Africa,3.44,4.62,34.3
19,Comoros,Sub-Saharan Africa,3.55,3.84,8.2
101,Tanzania,Sub-Saharan Africa,3.62,4.10,13.3
32,Ethiopia,Sub-Saharan Africa,3.63,4.35,19.8


,Country name,Regional indicator,Life Ladder,Predictions,Ecart en %
63,Lithuania,Central and Eastern Europe,7.04,5.90,-16.2
22,Costa Rica,Latin America and Caribbean,7.08,6.66,-5.9
64,Luxembourg,Western Europe,7.23,6.98,-3.5
83,Norway,Western Europe,7.30,7.40,1.4
78,Netherlands,Western Europe,7.39,7.07,-4.3
98,Sweden,Western Europe,7.43,7.48,0.7
45,Iceland,Western Europe,7.45,7.40,-0.7
26,Denmark,Western Europe,7.55,7.43,-1.6
50,Israel,Middle East and North Africa,7.66,6.80,-11.2
33,Finland,Western Europe,7.73,7.52,-2.7


In [134]:
display(df_2022_continents.sort_values('Ecart en %',ascending=False).head(10))
display(df_2022_continents.sort_values('Ecart en %',ascending=True).head(10))

,Country name,Regional indicator,Life Ladder,Predictions,Ecart en %
61,Lebanon,Middle East and North Africa,2.35,3.89,65.5
93,Sierra Leone,Sub-Saharan Africa,2.56,3.55,38.7
10,Botswana,Sub-Saharan Africa,3.44,4.62,34.3
21,Congo (Kinshasa),Sub-Saharan Africa,3.21,4.15,29.3
113,Zimbabwe,Sub-Saharan Africa,3.30,4.10,24.2
32,Ethiopia,Sub-Saharan Africa,3.63,4.35,19.8
6,Bangladesh,South Asia,3.41,4.08,19.6
101,Tanzania,Sub-Saharan Africa,3.62,4.10,13.3
66,Malawi,Sub-Saharan Africa,3.36,3.71,10.4
39,Ghana,Sub-Saharan Africa,4.19,4.61,10.0


,Country name,Regional indicator,Life Ladder,Predictions,Ecart en %
20,Congo (Brazzaville),Sub-Saharan Africa,5.81,4.37,-24.8
77,Nepal,South Asia,5.47,4.24,-22.5
42,Guinea,Sub-Saharan Africa,5.32,4.15,-22.0
63,Lithuania,Central and Eastern Europe,7.04,5.90,-16.2
71,Mexico,Latin America and Caribbean,7.04,5.91,-16.1
80,Nicaragua,Latin America and Caribbean,6.39,5.37,-16.0
73,Mongolia,East Asia,5.79,4.88,-15.7
92,Senegal,Sub-Saharan Africa,4.91,4.14,-15.7
30,El Salvador,Latin America and Caribbean,6.49,5.49,-15.4
112,Vietnam,Southeast Asia,6.27,5.37,-14.4


In [135]:
'''
On pourrait se demander si lorsqu'il y a des valeurs un peu plus extremes, le continent n'entraîne pas le modele 
vers une réponse plus cohérente en fonction du continents. 
On réalise donc un essai sur les mêmes données amputées de la variable continent.
'''

"\nOn pourrait se demander si lorsqu'il y a des valeurs un peu plus extremes, le continent n'entraîne pas le modele \nvers une réponse plus cohérente en fonction du continents. \nOn réalise donc un essai sur les mêmes données amputées de la variable continent.\n"

In [136]:
# Séparation des variables explicatives de la variable cible et suppression des noms de pays.
X_test2022_no_continents=df_2022.drop(['Life Ladder','Country name','Regional indicator'],axis=1)
y_test2022_no_continents=df_2022['Life Ladder']

#On normalise les variables explicatives
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
cols=['Log GDP per capita','Social support','Healthy life expectancy at birth','Freedom to make life choices','Generosity',
      'Perceptions of corruption','Positive affect','Negative affect']
X_test2022_no_continents[cols]=scaler.fit_transform(X_test2022_no_continents[cols])

#X_test2022_no_continents et y_test2022_no_continents sont alors respectivements nos variables explicatives et notre variable cible de notre jeu de test

In [137]:
# Suppression variables des continents du jeu de données 2021.
X_train2021_no_continents=X_train2021.drop(["Regional indicator_Commonwealth of Independent States"
                              ,"Regional indicator_East Asia"
                              ,"Regional indicator_Latin America and Caribbean"
                              ,"Regional indicator_Middle East and North Africa"
                              ,"Regional indicator_North America and ANZ"
                              ,"Regional indicator_South Asia"
                              ,"Regional indicator_Southeast Asia"
                              ,"Regional indicator_Sub-Saharan Africa"
                              ,"Regional indicator_Western Europe"],axis=1)

#On normalise les variables explicatives
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
cols=['Log GDP per capita','Social support','Healthy life expectancy at birth','Freedom to make life choices','Generosity',
      'Perceptions of corruption','Positive affect','Negative affect']
X_train2021_no_continents[cols]=scaler.fit_transform(X_train2021_no_continents[cols])

In [138]:
from sklearn.ensemble import RandomForestRegressor
rf_model = RandomForestRegressor(max_depth=20,max_features='sqrt',min_samples_leaf=1,min_samples_split=2,n_estimators=300)

rf_model.fit(X_train2021_no_continents,y_train2021)

print("Score sur jeu d'entrainement 2021 :",rf_model.score(X_train2021_no_continents,y_train2021))
print("Score sur jeu de test 2022 :",rf_model.score(X_test2022_no_continents,y_test2022))

Score sur jeu d'entrainement 2021 : 0.9833334612695842
Score sur jeu de test 2022 : 0.7438413072477355


In [141]:
# Réalisation d'une prédiction du Life Ladder de 2022 par pays grâce au modèle Random forest.
y_predic2022_no_continents = rf_model.predict(X_test2022_no_continents)

# Arrondissement de la prédictions à 2 décimales.
import numpy as np
y_predic2022_no_continents=np.round(y_predic2022_no_continents,2)

# Copie du jeu de donnée 2022 et suppression des continents.
df_2022_no_continents = df_2022.copy()

# Ajout d'une colonne avec les prédictions au jeu de données df_2022.
df_2022_no_continents['Predictions']=y_predic2022_no_continents
df_2022_no_continents=df_2022_no_continents.drop(cols,axis=1)

# Ajout d'une colonne pour représenter les variations en pourcentages.
df_2022_no_continents['Ecart en %']=np.round((df_2022_no_continents['Predictions']-df_2022_no_continents['Life Ladder'])/df_2022_no_continents['Life Ladder']*100,1)

In [145]:
df_2022_no_continents

,Country name,Regional indicator,Life Ladder,Predictions,Ecart en %
1,Albania,Central and Eastern Europe,5.21,4.93,-5.4
2,Argentina,Latin America and Caribbean,6.26,6.06,-3.2
3,Armenia,Commonwealth of Independent States,5.38,4.96,-7.8
4,Australia,North America and ANZ,7.04,6.90,-2.0
5,Austria,Western Europe,7.00,6.72,-4.0
...,...,...,...,...,...
108,United States,North America and ANZ,6.69,6.10,-8.8
109,Uruguay,Latin America and Caribbean,6.67,6.29,-5.7
110,Uzbekistan,Commonwealth of Independent States,6.02,5.20,-13.6
112,Vietnam,Southeast Asia,6.27,5.27,-15.9


In [144]:
df_2022_no_continents.drop('Country name',axis=1).groupby('Regional indicator').agg('mean').sort_values('Ecart en %',ascending=True)

#Grâce à ce df on s'aperçoit que notre modele prédit plutot bien, il a un peu plus de mal a prédire les valeurs extremes

,Life Ladder,Predictions,Ecart en %
Regional indicator,,,
Latin America and Caribbean,6.211111,5.540556,-10.688889
Commonwealth of Independent States,5.592500,5.116250,-8.275000
Southeast Asia,5.530000,5.115000,-6.600000
East Asia,5.973333,5.623333,-6.000000
Central and Eastern Europe,6.044545,5.784545,-3.990909
North America and ANZ,6.907500,6.685000,-3.275000
Western Europe,6.893333,6.781667,-1.627778
South Asia,4.270000,4.103333,-0.633333
Sub-Saharan Africa,4.289643,4.277857,2.032143


In [146]:
display(df_2022_no_continents.sort_values('Life Ladder',ascending=True).head(10))
display(df_2022_no_continents.sort_values('Life Ladder',ascending=True).tail(10))

,Country name,Regional indicator,Life Ladder,Predictions,Ecart en %
61,Lebanon,Middle East and North Africa,2.35,4.01,70.6
93,Sierra Leone,Sub-Saharan Africa,2.56,3.36,31.2
21,Congo (Kinshasa),Sub-Saharan Africa,3.21,4.09,27.4
113,Zimbabwe,Sub-Saharan Africa,3.30,4.15,25.8
66,Malawi,Sub-Saharan Africa,3.36,3.67,9.2
6,Bangladesh,South Asia,3.41,3.95,15.8
10,Botswana,Sub-Saharan Africa,3.44,4.63,34.6
19,Comoros,Sub-Saharan Africa,3.55,3.85,8.5
101,Tanzania,Sub-Saharan Africa,3.62,4.22,16.6
32,Ethiopia,Sub-Saharan Africa,3.63,4.26,17.4


,Country name,Regional indicator,Life Ladder,Predictions,Ecart en %
63,Lithuania,Central and Eastern Europe,7.04,5.93,-15.8
22,Costa Rica,Latin America and Caribbean,7.08,6.64,-6.2
64,Luxembourg,Western Europe,7.23,6.84,-5.4
83,Norway,Western Europe,7.30,7.42,1.6
78,Netherlands,Western Europe,7.39,7.08,-4.2
98,Sweden,Western Europe,7.43,7.35,-1.1
45,Iceland,Western Europe,7.45,7.41,-0.5
26,Denmark,Western Europe,7.55,7.31,-3.2
50,Israel,Middle East and North Africa,7.66,6.52,-14.9
33,Finland,Western Europe,7.73,7.40,-4.3


In [148]:
display(df_2022_no_continents.sort_values('Ecart en %',ascending=False).head(10))
display(df_2022_no_continents.sort_values('Ecart en %',ascending=True).head(10))

,Country name,Regional indicator,Life Ladder,Predictions,Ecart en %
61,Lebanon,Middle East and North Africa,2.35,4.01,70.6
10,Botswana,Sub-Saharan Africa,3.44,4.63,34.6
93,Sierra Leone,Sub-Saharan Africa,2.56,3.36,31.2
21,Congo (Kinshasa),Sub-Saharan Africa,3.21,4.09,27.4
113,Zimbabwe,Sub-Saharan Africa,3.30,4.15,25.8
32,Ethiopia,Sub-Saharan Africa,3.63,4.26,17.4
101,Tanzania,Sub-Saharan Africa,3.62,4.22,16.6
6,Bangladesh,South Asia,3.41,3.95,15.8
39,Ghana,Sub-Saharan Africa,4.19,4.72,12.6
55,Jordan,Middle East and North Africa,4.36,4.78,9.6


,Country name,Regional indicator,Life Ladder,Predictions,Ecart en %
20,Congo (Brazzaville),Sub-Saharan Africa,5.81,4.38,-24.6
71,Mexico,Latin America and Caribbean,7.04,5.40,-23.3
77,Nepal,South Asia,5.47,4.24,-22.5
30,El Salvador,Latin America and Caribbean,6.49,5.03,-22.5
80,Nicaragua,Latin America and Caribbean,6.39,4.95,-22.5
42,Guinea,Sub-Saharan Africa,5.32,4.21,-20.9
41,Guatemala,Latin America and Caribbean,6.15,4.89,-20.5
43,Honduras,Latin America and Caribbean,5.93,4.78,-19.4
9,Bolivia,Latin America and Caribbean,5.93,4.87,-17.9
37,Georgia,Commonwealth of Independent States,5.29,4.37,-17.4


In [163]:
# Pour terminer, on aggrège les deux jeux de données.
df_2022 = df_2022_continents
df_2022

,Country name,Regional indicator,Life Ladder,Predictions,Ecart en %
1,Albania,Central and Eastern Europe,5.21,4.94,-5.2
2,Argentina,Latin America and Caribbean,6.26,6.11,-2.4
3,Armenia,Commonwealth of Independent States,5.38,4.98,-7.4
4,Australia,North America and ANZ,7.04,7.03,-0.1
5,Austria,Western Europe,7.00,6.75,-3.6
...,...,...,...,...,...
108,United States,North America and ANZ,6.69,6.12,-8.5
109,Uruguay,Latin America and Caribbean,6.67,6.33,-5.1
110,Uzbekistan,Commonwealth of Independent States,6.02,5.41,-10.1
112,Vietnam,Southeast Asia,6.27,5.37,-14.4


In [164]:
# Changement de nom des colonnes
new_names =  {'Predictions' : 'Prédictions avec continents',
              'Ecart en %'  : 'Écart en % avec continents'}

df_2022 = df_2022.rename(new_names, axis = 1)
df_2022

,Country name,Regional indicator,Life Ladder,Prédictions avec continents,Écart en % avec continents
1,Albania,Central and Eastern Europe,5.21,4.94,-5.2
2,Argentina,Latin America and Caribbean,6.26,6.11,-2.4
3,Armenia,Commonwealth of Independent States,5.38,4.98,-7.4
4,Australia,North America and ANZ,7.04,7.03,-0.1
5,Austria,Western Europe,7.00,6.75,-3.6
...,...,...,...,...,...
108,United States,North America and ANZ,6.69,6.12,-8.5
109,Uruguay,Latin America and Caribbean,6.67,6.33,-5.1
110,Uzbekistan,Commonwealth of Independent States,6.02,5.41,-10.1
112,Vietnam,Southeast Asia,6.27,5.37,-14.4


In [165]:
df_2022['Prédictions sans continents'] = df_2022_no_continents['Predictions']
df_2022['Écart en % sans continents'] = df_2022_no_continents['Ecart en %']
df_2022

,Country name,Regional indicator,Life Ladder,Prédictions avec continents,Écart en % avec continents,Prédictions sans continents,Écart en % sans continents
1,Albania,Central and Eastern Europe,5.21,4.94,-5.2,4.93,-5.4
2,Argentina,Latin America and Caribbean,6.26,6.11,-2.4,6.06,-3.2
3,Armenia,Commonwealth of Independent States,5.38,4.98,-7.4,4.96,-7.8
4,Australia,North America and ANZ,7.04,7.03,-0.1,6.90,-2.0
5,Austria,Western Europe,7.00,6.75,-3.6,6.72,-4.0
...,...,...,...,...,...,...,...
108,United States,North America and ANZ,6.69,6.12,-8.5,6.10,-8.8
109,Uruguay,Latin America and Caribbean,6.67,6.33,-5.1,6.29,-5.7
110,Uzbekistan,Commonwealth of Independent States,6.02,5.41,-10.1,5.20,-13.6
112,Vietnam,Southeast Asia,6.27,5.37,-14.4,5.27,-15.9


In [166]:
# Ajout d'une colonne pour représenter les différences de pourcentages.
df_2022['Différences valeurs écarts'] = df_2022['Prédictions sans continents']-df_2022['Prédictions avec continents']
df_2022

,Country name,Regional indicator,Life Ladder,Prédictions avec continents,Écart en % avec continents,Prédictions sans continents,Écart en % sans continents,Différences valeurs écarts
1,Albania,Central and Eastern Europe,5.21,4.94,-5.2,4.93,-5.4,-0.01
2,Argentina,Latin America and Caribbean,6.26,6.11,-2.4,6.06,-3.2,-0.05
3,Armenia,Commonwealth of Independent States,5.38,4.98,-7.4,4.96,-7.8,-0.02
4,Australia,North America and ANZ,7.04,7.03,-0.1,6.90,-2.0,-0.13
5,Austria,Western Europe,7.00,6.75,-3.6,6.72,-4.0,-0.03
...,...,...,...,...,...,...,...,...
108,United States,North America and ANZ,6.69,6.12,-8.5,6.10,-8.8,-0.02
109,Uruguay,Latin America and Caribbean,6.67,6.33,-5.1,6.29,-5.7,-0.04
110,Uzbekistan,Commonwealth of Independent States,6.02,5.41,-10.1,5.20,-13.6,-0.21
112,Vietnam,Southeast Asia,6.27,5.37,-14.4,5.27,-15.9,-0.10


In [167]:
# Ajout d'une colonne pour représenter les différences de pourcentages.
df_2022['% variations écarts'] = df_2022['Écart en % sans continents']-df_2022['Écart en % avec continents']
df_2022

,Country name,Regional indicator,Life Ladder,Prédictions avec continents,Écart en % avec continents,Prédictions sans continents,Écart en % sans continents,Différences valeurs écarts,% variations écarts
1,Albania,Central and Eastern Europe,5.21,4.94,-5.2,4.93,-5.4,-0.01,-0.2
2,Argentina,Latin America and Caribbean,6.26,6.11,-2.4,6.06,-3.2,-0.05,-0.8
3,Armenia,Commonwealth of Independent States,5.38,4.98,-7.4,4.96,-7.8,-0.02,-0.4
4,Australia,North America and ANZ,7.04,7.03,-0.1,6.90,-2.0,-0.13,-1.9
5,Austria,Western Europe,7.00,6.75,-3.6,6.72,-4.0,-0.03,-0.4
...,...,...,...,...,...,...,...,...,...
108,United States,North America and ANZ,6.69,6.12,-8.5,6.10,-8.8,-0.02,-0.3
109,Uruguay,Latin America and Caribbean,6.67,6.33,-5.1,6.29,-5.7,-0.04,-0.6
110,Uzbekistan,Commonwealth of Independent States,6.02,5.41,-10.1,5.20,-13.6,-0.21,-3.5
112,Vietnam,Southeast Asia,6.27,5.37,-14.4,5.27,-15.9,-0.10,-1.5


In [168]:
df_2022.describe()

,Life Ladder,Prédictions avec continents,Écart en % avec continents,Prédictions sans continents,Écart en % sans continents,Différences valeurs écarts,% variations écarts
count,103.000000,103.000000,103.000000,103.000000,103.000000,103.000000,103.000000
mean,5.591359,5.401650,-1.906796,5.352524,-2.675728,-0.049126,-0.768932
std,1.229449,1.068112,12.520636,1.055177,13.202395,0.157661,2.801331
min,2.350000,3.550000,-24.800000,3.360000,-24.600000,-0.580000,-9.600000
25%,4.675000,4.470000,-9.200000,4.405000,-9.150000,-0.100000,-1.650000
50%,5.890000,5.410000,-3.300000,5.260000,-4.000000,-0.010000,-0.200000
75%,6.465000,6.115000,1.300000,6.160000,1.650000,0.035000,0.650000
max,7.730000,7.520000,65.500000,7.420000,70.600000,0.430000,7.500000


In [ ]:
# On constate que les prédictions sont moins précises si l'on enlève les données relatives aux continents.